In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../Includes/configuration"

In [0]:
%run "../Includes/common_functions"

In [0]:
race_results_list = spark.read\
    .format("delta").load((f"{presentation_folder_path}/race_results"))\
    .withColumnRenamed("result_file_date", "file_date")\
    .filter(f"file_date = '{v_file_date}'")\
    .select("race_year")\
    .distinct()\
    .collect()

In [0]:
race_year_list = []

for race_year in race_results_list:
    race_year_list.append(race_year.race_year)

In [0]:
df = spark.read\
     .format("delta").load((f"{presentation_folder_path}/race_results"))\
     .filter(col("race_year").isin(race_year_list))

In [0]:
constructor_standings_df = df\
    .groupBy("race_year", "team")\
        .agg(sum("points").alias("total_points"),
                 count(when(col("position") == 1, True)).alias("wins"))

In [0]:
window_spec = Window.partitionBy("race_year").orderBy(col("total_points").desc(), col("wins").desc())

In [0]:
df_final = constructor_standings_df.withColumn("rank", rank().over(window_spec))


In [0]:
#overwrite_partition(df_final, "f1_presentation", "constructor_standings", "race_year")

In [0]:
merge_condition = "tgt.team = src.team AND tgt.race_year = src.race_year"
merge_delta_data(df_final, "f1_presentation", "constructor_standings", presentation_folder_path, merge_condition, "race_year")

In [0]:
%sql
select * from f1_presentation.constructor_standings
where race_year = 2020
order by wins desc

race_year,team,total_points,wins,rank
2020,Mercedes,573.0,13,1
2020,Red Bull,319.0,2,2
2020,Racing Point,210.0,1,3
2020,AlphaTauri,107.0,1,7
2020,Alfa Romeo,8.0,0,8
2020,Renault,181.0,0,5
2020,Ferrari,131.0,0,6
2020,McLaren,202.0,0,4
2020,Haas F1 Team,3.0,0,9
2020,Williams,0.0,0,10
